In [ ]:
#import voor alle benodigde libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
import seaborn as sns
import plotly.express as px
from sodapy import Socrata
import folium

In [ ]:
#Inladen API chargemap
response = requests.get("https://api.openchargemap.io/v3/poi/?output=json&countrycode=NL&maxresults=8000&compact=true&verbose=false&key=6686970f-0aa6-4cb5-ae81-cf8f6389a816")

#Omzetten naar dictionary
responsejson  = response.json()

#Dataframe bevat kolom die een list zijn. 
#Met json_normalize zet je de eerste kolom om naar losse kolommen
charge_map = pd.json_normalize(responsejson)

charge_map_df = pd.DataFrame.from_records(charge_map)

#Dropping unnecessary columns:
charge_map_df = charge_map_df.drop(columns=['IsRecentlyVerified', 'UUID', 'DataProviderID', 'AddressInfo.CountryID', 
                            'AddressInfo.DistanceUnit', 'AddressInfo.RelatedURL', 'AddressInfo.ContactTelephone1', 
                            'AddressInfo.ContactEmail', 'AddressInfo.ContactTelephone2', 'OperatorsReference', 
                            'DataProvidersReference', 'GeneralComments', 'AddressInfo.AddressLine2',
                            'AddressInfo.AccessComments'
                            ])

#Renaming column names:
charge_map_df.rename(columns={'AddressInfo.Latitude':'LAT', 'AddressInfo.Longitude':'LON', 
                              'AddressInfo.Postcode':'Postcode', 'AddressInfo.ID':'Address_ID', 
                              'AddressInfo.Title':'Address_Title', 'AddressInfo.AddressLine1':'Address_Line1',
                              'AddressInfo.StateOrProvince':'Address_StateOrProvince', 
                              'AddressInfo.Town':'Address_Town'},
                              inplace=True)                            


In [4]:
charge_map_df.describe()
#We zien dat LAT een te lage min value heeft, outliers die gooien we eruit

,ID,DataQualityLevel,Address_ID,Postcode,LAT,LON
count,7810.000000,7809.000000,7810.000000,7810.000000,7810.000000,7810.000000
mean,50247.053009,2.850173,50591.794622,4224.356978,52.079303,5.122159
std,26858.926529,0.526537,26863.169212,2314.393877,0.445916,0.694530
min,4621.000000,1.000000,4566.000000,1011.000000,50.766217,3.382141
25%,35347.250000,3.000000,35693.250000,2512.000000,51.837373,4.584757
50%,38600.000000,3.000000,38946.000000,3581.000000,52.069113,5.005548
75%,63330.750000,3.000000,63676.750000,5709.000000,52.322024,5.612725
max,258262.000000,3.000000,258646.000000,9991.000000,53.478088,7.189636


In [ ]:
#Drop de outlier
charge_map_df = charge_map_df.drop(charge_map_df[charge_map_df['LAT'] < 50].index)

In [ ]:
charge_map_df.isna().sum()
#We zien dat UsageTypeID en Address_StateOrProvince meer dan 80% NaN hebben, die filteren we eruit.
charge_map_df = charge_map_df.drop(columns=['UsageTypeID', 'Address_StateOrProvince'])

In [ ]:
#We hebben bepaald dat de data in de volgende columns niet nodig gaat zijn:
charge_map_df = charge_map_df.drop(columns=['OperatorID', 'Connections', 'StatusTypeID', 'SubmissionStatusTypeID'])

In [ ]:
#Dropping empty values
charge_map_df = charge_map_df.drop(charge_map_df[charge_map_df['Postcode']==''].index)
charge_map_df = charge_map_df.drop(charge_map_df[charge_map_df['Postcode']=='XG'].index)
charge_map_df = charge_map_df.dropna(subset=['Postcode'])

#Transforming the Postcode to Integers
charge_map_df['Postcode'] = charge_map_df['Postcode'].astype(str)
charge_map_df['Postcode'] = charge_map_df['Postcode'].str[0:4]
charge_map_df['Postcode'] = charge_map_df['Postcode'].astype(int)

#Dropping incorrect postcodes
charge_map_df = charge_map_df.drop(charge_map_df[charge_map_df['Postcode'] < 1000].index)

In [ ]:
#Creating address types to make color coding possible.
#Observerd types:
#-Restaurants   = MCDonald's / Mc Donald's,
#-Recreatie     = hotel, Gelredome
#-Parkeren      = P+R, parkeerterrein
#-Tankstations  = Fastned, Supercharger (Tesla),

#Recreation type
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Hotel', case=False), 'Address_Type'] = 'Recreatie'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Gelredome', case=False), 'Address_Type'] = 'Recreatie'

#Tankstation type
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Fastned', case=False), 'Address_Type'] = 'Tankstation'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Supercharger', case=False), 'Address_Type'] = 'Tankstation'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Shell', case=False), 'Address_Type'] = 'Tankstation'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('IONITY', case=False), 'Address_Type'] = 'Tankstation'

#Restaurants type
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Mc Donald', case=False), 'Address_Type'] = 'Restaurant'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('McDonald', case=False), 'Address_Type'] = 'Restaurant'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Restaurant', case=False), 'Address_Type'] = 'Restaurant'

#Parkeerplaats type
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Parkeer', case=False), 'Address_Type'] = 'Parkeerterrein'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Garage', case=False), 'Address_Type'] = 'Parkeerterrein'

#Filling in the NaNs with 'Straat'
charge_map_df['Address_Type'].fillna('Straat', inplace=True)

In [ ]:
#Dropping typos
#De typos zijn handmatig bepaald door te kijken naar de indexen van outliers in de folium map
typos = [364, 4947, 1414, 1203, 5479, 1351, 2387, 1242, 479, 1882, 1597, 2413, 2064, 1670, 1941, 1982, 4094, 2831, 7198]
charge_map_df = charge_map_df.drop(index= typos)

In [ ]:
#Changing the column 'NumberOfPoints' to string
charge_map_df['NumberOfPoints'] = charge_map_df['NumberOfPoints'].astype(int).astype(str)


In [3]:
#Het downloaden van de schone CSV
charge_map_df.to_csv(r'C:\School\Y3\Semester 2 - Minor\Block 1\Case3\chargemap.csv')

print('File has been saved')

KeyboardInterrupt: 